In [1]:
def read_file(filepath):
  with open(filepath) as f:
    str_text = f.read()
    
    return str_text

In [8]:
filename = "/Users/test/Downloads/UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt"

In [ ]:
read_file()

In [2]:
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
nlp.max_length = 1198623

In [22]:
def separate_punc(doc_text):
  return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [9]:
d = read_file(filename)

In [23]:
tokens = separate_punc(d)

In [24]:
len(tokens)

11338

In [25]:
train_len = 25 +1

text_seq = []

for i in range(train_len, len(tokens)):
  seq = tokens[i-train_len:i]
  text_seq.append(seq)

In [26]:
from keras.preprocessing.text import Tokenizer

In [27]:
tokenizer = Tokenizer()

In [28]:
tokenizer.fit_on_texts(text_seq)
seq = tokenizer.texts_to_sequences(text_seq)
seq

[[956,
  14,
  263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24],
 [14,
  263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957],
 [263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5],
 [51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60],
 [261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60,
  5],
 [408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60,
  5,
  5

In [35]:
vocab_size = len(tokenizer.word_counts)

In [30]:
import numpy as np

seq = np.array(seq)

In [31]:
from keras.utils import to_categorical

In [61]:
X = seq[:,:-1]

In [86]:
y = seq[:,-1]

In [87]:
y = to_categorical(y, num_classes=vocab_size+1)

In [38]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [88]:
def create_model(vocab_size, seq_len):
  model = Sequential()
  model.add(Embedding(vocab_size, seq_len, input_length=seq_len))
  model.add(LSTM(50, return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(vocab_size, activation='softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  model.summary()
  return model

In [89]:
model = create_model(vocab_size+1,X.shape[1])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 25, 25)            68000     
                                                                 
 lstm_18 (LSTM)              (None, 25, 50)            15200     
                                                                 
 lstm_19 (LSTM)              (None, 50)                20200     
                                                                 
 dense_18 (Dense)            (None, 50)                2550      
                                                                 
 dense_19 (Dense)            (None, 2720)              138720    
                                                                 
Total params: 244,670
Trainable params: 244,670
Non-trainable params: 0
_________________________________________________________________


In [90]:
model.fit(X,y,batch_size=128,epochs=50, verbose=1)

Epoch 1/300


89/89 [==============================] - 4s 29ms/step - loss: 6.9899 - accuracy: 0.0334
Epoch 2/300
89/89 [==============================] - 6s 63ms/step - loss: 6.3775 - accuracy: 0.0529
Epoch 3/300
89/89 [==============================] - 5s 56ms/step - loss: 6.3487 - accuracy: 0.0529
Epoch 4/300
89/89 [==============================] - 3s 39ms/step - loss: 6.3147 - accuracy: 0.0529
Epoch 5/300
89/89 [==============================] - 3s 32ms/step - loss: 6.1913 - accuracy: 0.0529
Epoch 6/300
89/89 [==============================] - 2s 27ms/step - loss: 6.1146 - accuracy: 0.0529
Epoch 7/300
89/89 [==============================] - 3s 29ms/step - loss: 6.0712 - accuracy: 0.0493
Epoch 8/300
89/89 [==============================] - 3s 29ms/step - loss: 6.0196 - accuracy: 0.0577
Epoch 9/300
89/89 [==============================] - 3s 28ms/step - loss: 5.9531 - accuracy: 0.0653
Epoch 10/300
89/89 [==============================] - 2s 27ms/step - loss: 5.8790 - accuracy: 0.0652
Epoch 11/30

KeyboardInterrupt: 

In [ ]:
from pickle import dump


model.save("my_model.h5")

dump(tokenizer, open("my_tokenier", 'wb'))